In [8]:
#Run this if you want to force to use CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# CNN

In [9]:
#!/usr/bin/env python3
import os

from tensorflow.keras.models import load_model
from keras.utils.vis_utils import plot_model  # <-- importante en algunas versiones

from qkeras.utils import _add_supported_quantized_objects

# Registrar capas cuantizadas
co = {}
_add_supported_quantized_objects(co)

# === Ruta local ===
MODEL_DIR = "models/CNN/"

model_files = {
    "studentModelMnistCNN.h5",
    "teacherModelMnistCNN.h5",
    "teacherModelQATMnistCNN.h5"
}

# === Crear carpeta outputs si no existe ===
OUTPUT_DIR = "model_plots/CNN"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def safe_load_model(path):
    try:
        model = load_model(path, custom_objects=co)
        return model, None
    except Exception as e:
        return None, str(e)

for filename in model_files:
    path = os.path.join(MODEL_DIR, filename)
    print(f"\nCargando modelo: {path}")

    model, err = safe_load_model(path)
    if err:
        print(f"Error al cargar {filename}: {err}")
        continue

    outpath = os.path.join(OUTPUT_DIR, filename.replace(".h5", ".png"))
    print(f"Generando diagrama: {outpath}")

    plot_model(
        model,
        to_file=outpath,
        show_shapes=True,
        show_layer_names=True,
        expand_nested=True,
        dpi=120
    )

print("\nListo ✔ Diagrama de modelos generado en carpeta 'model_plots'")



Cargando modelo: models/CNN/studentModelMnistCNN.h5


Generando diagrama: model_plots/CNN/studentModelMnistCNN.png

Cargando modelo: models/CNN/teacherModelQATMnistCNN.h5
Generando diagrama: model_plots/CNN/teacherModelQATMnistCNN.png

Cargando modelo: models/CNN/teacherModelMnistCNN.h5
Generando diagrama: model_plots/CNN/teacherModelMnistCNN.png

Listo ✔ Diagrama de modelos generado en carpeta 'model_plots'


In [1]:
#!/usr/bin/env python3
"""
Comparación entre modelos Teacher y Student para MNIST-CNN
Métricas:
- Accuracy
- Loss
- Parámetros totales
- Sparsity (fracción de ceros)
- Tamaño en disco
- FLOPs aproximados
- Latencia por muestra
- Speedup relativo
- Comparación porcentual con Teacher
"""

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist

# --- QKeras imports ---
from qkeras.utils import _add_supported_quantized_objects

# Registrar objetos cuantizados
co = {}
_add_supported_quantized_objects(co)


# === RUTA LOCAL (ajustada a tu PC) ===
MODEL_DIR = "models/CNN/"

model_files = {
    "student": os.path.join(MODEL_DIR, "studentModelMnistCNN.h5"),
    "teacher": os.path.join(MODEL_DIR, "teacherModelMnistCNN.h5"),
}


# === Carga MNIST ===
(_, _), (x_test, y_test) = mnist.load_data()

# Normalización
x_test = x_test.astype("float32") / 255.0

# Añadir canal: (N, 28, 28, 1)
x_test = x_test[..., tf.newaxis]


# === Función de carga segura ===
def safe_load_model(path):
    try:
        model = load_model(path, custom_objects=co)
        return model, None
    except Exception as e:
        return None, str(e)


# === FLOPs aproximados (Keras) ===
def count_flops(model):
    """
    Cálculo aproximado usando TF2 profiler.
    No es exacto pero sirve para comparaciones relativas.
    """
    try:
        concrete = tf.function(model).get_concrete_function(
            tf.TensorSpec([1, 28, 28, 1], model.inputs[0].dtype)
        )
        frozen_func, graph_def = tf.graph_util.convert_variables_to_constants_v2_as_graph(concrete)
        flops = tf.compat.v1.profiler.profile(
            graph=frozen_func.graph,
            options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        )
        return flops.total_float_ops
    except:
        return None


results = []

for name, path in model_files.items():

    info = {"model": name, "path": path, "exists": os.path.exists(path)}

    if not os.path.exists(path):
        info["error"] = "NOT FOUND"
        results.append(info)
        continue

    info["filesize_bytes"] = os.path.getsize(path)

    # Tamaño del modelo
    info["filesize_mb"] = info["filesize_bytes"] / (1024 * 1024)


    model, err = safe_load_model(path)
    if err:
        info["error"] = f"ERROR: {err}"
        results.append(info)
        continue

    info["loaded"] = True

    # Parámetros
    info["params_count"] = model.count_params()

    # Sparsity
    weights = model.get_weights()
    total = sum(w.size for w in weights)
    zeros = sum(np.sum(w == 0.0) for w in weights)
    info["weights_total"] = total
    info["weights_zeros"] = zeros
    info["sparsity_fraction"] = zeros / total

    # Evaluación
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    info["eval_loss"] = loss
    info["eval_accuracy"] = acc

    # FLOPs
    flops = count_flops(model)
    info["flops_total"] = flops

    # Latencia
    model.predict(x_test[:64], verbose=0)  # warm up
    t0 = time.time()
    model.predict(x_test, batch_size=256, verbose=0)
    t1 = time.time()

    info["inference_time_total_s"] = t1 - t0
    info["inference_time_per_sample_ms"] = (t1 - t0) / len(x_test) * 1000

    results.append(info)


# === DataFrame ===
df = pd.DataFrame(results)

# referencia Teacher para porcentajes
teacher = df[df.model == "teacher"].iloc[0]

# comparación relativa
df["params_ratio_vs_teacher"] = df.params_count / teacher.params_count
df["acc_drop_vs_teacher"] = df.eval_accuracy - teacher.eval_accuracy
df["speedup_vs_teacher"] = teacher.inference_time_per_sample_ms / df.inference_time_per_sample_ms
df["sparsity_vs_teacher"] = df.sparsity_fraction - teacher.sparsity_fraction
# Comparación relativa frente al teacher
teacher = df[df.model == "teacher"].iloc[0]

df["size_ratio_vs_teacher"] = df.filesize_mb / teacher.filesize_mb


df.to_csv("mnist_summaryCNN.csv", index=False)

print("\n=== RESUMEN MODELOS CNN ===")

print(df.to_string(index=False))

print("\nCSV: mnist_summaryCNN.csv")


2025-12-15 17:19:55.662263: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 17:19:56.143586: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 17:19:56.145494: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-15 17:19:57.845260: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-12-15 17:20:01.494283: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Do


=== RESUMEN MODELOS CNN ===
  model                               path  exists  filesize_bytes  filesize_mb  loaded  params_count  weights_total  weights_zeros  sparsity_fraction  eval_loss  eval_accuracy flops_total  inference_time_total_s  inference_time_per_sample_ms  params_ratio_vs_teacher  acc_drop_vs_teacher  speedup_vs_teacher  sparsity_vs_teacher  size_ratio_vs_teacher
student models/CNN/studentModelMnistCNN.h5    True           53992     0.051491    True          3816           3816           1852           0.485325   0.202003         0.9381        None                1.548102                      0.154810                 0.804046                -0.03            0.306342             0.485114               0.495449
teacher models/CNN/teacherModelMnistCNN.h5    True          108976     0.103928    True          4746           4746              1           0.000211   0.212673         0.9681        None                0.474249                      0.047425                 1.0000

# MLP

In [11]:
#!/usr/bin/env python3
import os

from tensorflow.keras.models import load_model
from keras.utils.vis_utils import plot_model  # <-- importante en algunas versiones

from qkeras.utils import _add_supported_quantized_objects

# Registrar capas cuantizadas
co = {}
_add_supported_quantized_objects(co)

# === Ruta local ===
MODEL_DIR = "models/MLP/"

model_files = {
    "studentModelMnistMLP.h5",
    "teacherModelMnistMLP.h5",
    "teacherModelQATMnistMLP.h5"
}

# === Crear carpeta outputs si no existe ===
OUTPUT_DIR = "model_plots/MLP"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def safe_load_model(path):
    try:
        model = load_model(path, custom_objects=co)
        return model, None
    except Exception as e:
        return None, str(e)

for filename in model_files:
    path = os.path.join(MODEL_DIR, filename)
    print(f"\nCargando modelo: {path}")

    model, err = safe_load_model(path)
    if err:
        print(f"Error al cargar {filename}: {err}")
        continue

    outpath = os.path.join(OUTPUT_DIR, filename.replace(".h5", ".png"))
    print(f"Generando diagrama: {outpath}")

    plot_model(
        model,
        to_file=outpath,
        show_shapes=True,
        show_layer_names=True,
        expand_nested=True,
        dpi=120
    )

print("\nListo ✔ Diagrama de modelos generado en carpeta 'model_plots'")



Cargando modelo: models/MLP/studentModelMnistMLP.h5


Generando diagrama: model_plots/MLP/studentModelMnistMLP.png

Cargando modelo: models/MLP/teacherModelMnistMLP.h5
Generando diagrama: model_plots/MLP/teacherModelMnistMLP.png

Cargando modelo: models/MLP/teacherModelQATMnistMLP.h5
Generando diagrama: model_plots/MLP/teacherModelQATMnistMLP.png

Listo ✔ Diagrama de modelos generado en carpeta 'model_plots'


In [2]:
#!/usr/bin/env python3
"""
Comparación entre modelos Teacher y Student para MNIST-CNN
Métricas:
- Accuracy
- Loss
- Parámetros totales
- Sparsity (fracción de ceros)
- Tamaño en disco
- FLOPs aproximados
- Latencia por muestra
- Speedup relativo
- Comparación porcentual con Teacher
"""

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist

# --- QKeras imports ---
from qkeras.utils import _add_supported_quantized_objects

# Registrar objetos cuantizados
co = {}
_add_supported_quantized_objects(co)


# === RUTA LOCAL (ajustada a tu PC) ===
MODEL_DIR = "models/MLP/"

model_files = {
    "student": os.path.join(MODEL_DIR, "studentModelMnistMLP.h5"),
    "teacher": os.path.join(MODEL_DIR, "teacherModelMnistMLP.h5"),
}


# === Carga MNIST ===
(_, _), (x_test, y_test) = mnist.load_data()

# Normalización
x_test = x_test.astype("float32") / 255.0

# Aplanar para MLP: (N, 784)
x_test = x_test.reshape(len(x_test), 28 * 28)



# === Función de carga segura ===
def safe_load_model(path):
    try:
        model = load_model(path, custom_objects=co)
        return model, None
    except Exception as e:
        return None, str(e)


# === FLOPs aproximados (Keras) ===
def count_flops(model):
    """
    Cálculo aproximado usando TF2 profiler.
    No es exacto pero sirve para comparaciones relativas.
    """
    try:
        concrete = tf.function(model).get_concrete_function(
            tf.TensorSpec([1, 28, 28, 1], model.inputs[0].dtype)
        )
        frozen_func, graph_def = tf.graph_util.convert_variables_to_constants_v2_as_graph(concrete)
        flops = tf.compat.v1.profiler.profile(
            graph=frozen_func.graph,
            options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        )
        return flops.total_float_ops
    except:
        return None


results = []

for name, path in model_files.items():

    info = {"model": name, "path": path, "exists": os.path.exists(path)}

    if not os.path.exists(path):
        info["error"] = "NOT FOUND"
        results.append(info)
        continue

    info["filesize_bytes"] = os.path.getsize(path)

    # Tamaño del modelo
    info["filesize_mb"] = info["filesize_bytes"] / (1024 * 1024)


    model, err = safe_load_model(path)
    if err:
        info["error"] = f"ERROR: {err}"
        results.append(info)
        continue

    info["loaded"] = True

    # Parámetros
    info["params_count"] = model.count_params()

    # Sparsity
    weights = model.get_weights()
    total = sum(w.size for w in weights)
    zeros = sum(np.sum(w == 0.0) for w in weights)
    info["weights_total"] = total
    info["weights_zeros"] = zeros
    info["sparsity_fraction"] = zeros / total

    # Evaluación
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    info["eval_loss"] = loss
    info["eval_accuracy"] = acc

    # FLOPs
    flops = count_flops(model)
    info["flops_total"] = flops

    # Latencia
    model.predict(x_test[:64], verbose=0)  # warm up
    t0 = time.time()
    model.predict(x_test, batch_size=256, verbose=0)
    t1 = time.time()

    info["inference_time_total_s"] = t1 - t0
    info["inference_time_per_sample_ms"] = (t1 - t0) / len(x_test) * 1000

    results.append(info)


# === DataFrame ===
df = pd.DataFrame(results)

# referencia Teacher para porcentajes
teacher = df[df.model == "teacher"].iloc[0]

# comparación relativa
df["params_ratio_vs_teacher"] = df.params_count / teacher.params_count
df["acc_drop_vs_teacher"] = df.eval_accuracy - teacher.eval_accuracy
df["speedup_vs_teacher"] = teacher.inference_time_per_sample_ms / df.inference_time_per_sample_ms
df["sparsity_vs_teacher"] = df.sparsity_fraction - teacher.sparsity_fraction
# Comparación relativa frente al teacher
teacher = df[df.model == "teacher"].iloc[0]

df["size_ratio_vs_teacher"] = df.filesize_mb / teacher.filesize_mb


df.to_csv("mnist_summaryMLP.csv", index=False)

print("\n=== RESUMEN MODELOS MLP ===")
print(df.to_string(index=False))




print("\nCSV: mnist_summaryCNN.csv")



=== RESUMEN MODELOS MLP ===
  model                               path  exists  filesize_bytes  filesize_mb  loaded  params_count  weights_total  weights_zeros  sparsity_fraction  eval_loss  eval_accuracy flops_total  inference_time_total_s  inference_time_per_sample_ms  params_ratio_vs_teacher  acc_drop_vs_teacher  speedup_vs_teacher  sparsity_vs_teacher  size_ratio_vs_teacher
student models/MLP/studentModelMnistMLP.h5    True           62816     0.059906    True          8847           8847           4347           0.491353   0.714022         0.7895        None                0.616044                      0.061604                  0.96678              -0.0958            0.329361             0.491353               0.425099
teacher models/MLP/teacherModelMnistMLP.h5    True          147768     0.140923    True          9151           9151              0           0.000000   0.403666         0.8853        None                0.202901                      0.020290                  1.000